In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/hvidsmen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/hvidsmen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/hvidsmen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Simple

In [5]:
batch_size = 64
latent_space = 128
learning_rate = 0.1

In [17]:
ae_weights = {
    'encoder_w': tf.Variable(tf.truncated_normal([784,latent_space],stddev=0.1)),
    'encoder_b': tf.Variable(tf.truncated_normal([latent_space],stddev=0.1)),
    'decoder_w':tf.Variable(tf.truncated_normal([latent_space,784],stddev=0.1)),
    'decoder_b':tf.Variable(tf.truncated_normal([784],stddev=0.1)),
}
ae_input = tf.placeholder(tf.float32,[batch_size,784])
hidden = tf.nn.sigmoid(tf.matmul(ae_input,ae_weights['encoder_w']) + ae_weights['encoder_b'])

visible_logits = tf.matmul(hidden, ae_weights['decoder_w']) + ae_weights['decoder_b']
visible = tf.nn.sigmoid(visible_logits)



ae_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits,labels=ae_input))
optimizer = tf.train.AdamOptimizer(learning_rate)
ae_op = optimizer.minimize(ae_cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        x_batch, _ = mnist.train.next_batch(batch_size)
        sess.run(ae_op,feed_dict={ae_input:x_batch})
        score = sess.run(ae_cost,feed_dict={ae_input:x_batch})
        if i%10==0:
            print(f'score = {score}')
    print(f'score = {score}')


score = 0.46477460861206055
score = 0.284026563167572
score = 0.27533456683158875
score = 0.2624067962169647
score = 0.25136396288871765
score = 0.2444850206375122
score = 0.24982768297195435
score = 0.2501866817474365
score = 0.25556135177612305
score = 0.25790929794311523
score = 0.25386902689933777


Разряженный автоинкодер

In [18]:
rho=0.05
beta = 1.0

ae_weights = {
    'encoder_w': tf.Variable(tf.truncated_normal([784,latent_space],stddev=0.1)),
    'encoder_b': tf.Variable(tf.truncated_normal([latent_space],stddev=0.1)),
    'decoder_w':tf.Variable(tf.truncated_normal([latent_space,784],stddev=0.1)),
    'decoder_b':tf.Variable(tf.truncated_normal([784],stddev=0.1)),
}
ae_input = tf.placeholder(tf.float32,[batch_size,784])
hidden = tf.nn.sigmoid(tf.matmul(ae_input,ae_weights['encoder_w']) + ae_weights['encoder_b'])

noise_hidden = tf.nn.relu(hidden-0.1) + 0.1
noise_visible = tf.nn.sigmoid(
    tf.matmul(noise_hidden, ae_weights['decoder_w']) + ae_weights['decoder_b']
)

deta_rho = tf.reduce_mean(hidden,0)
reg_cost= -tf.reduce_mean(tf.log(deta_rho/rho)*rho + tf.log(1-deta_rho/(1-rho)*(1-rho)))
visible_logits = tf.matmul(hidden, ae_weights['decoder_w']) + ae_weights['decoder_b']
visible = tf.nn.sigmoid(visible_logits)



ae_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits,labels=ae_input))
total_cost = reg_cost + ae_cost
optimizer = tf.train.AdamOptimizer(learning_rate)
ae_op = optimizer.minimize(total_cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        x_batch, _ = mnist.train.next_batch(batch_size)
        sess.run(ae_op,feed_dict={ae_input:x_batch})
        score = sess.run(ae_cost,feed_dict={ae_input:x_batch})
        if i%10==0:
            print(f'score = {score}')
    print(f'score = {score}')


score = 0.6553716659545898
score = 0.40043017268180847
score = 0.36065372824668884
score = 0.2787542939186096
score = 0.2691074013710022
score = 0.2762235403060913
score = 0.2645537853240967
score = 0.26074400544166565
score = 0.24193768203258514
score = 0.25175613164901733
score = 0.23993618786334991


Шумоподовляющий автокодировщик

In [22]:
rho=0.05
beta = 1.0

ae_weights = {
    'encoder_w': tf.Variable(tf.truncated_normal([784,latent_space],stddev=0.1)),
    'encoder_b': tf.Variable(tf.truncated_normal([latent_space],stddev=0.1)),
    'decoder_w':tf.Variable(tf.truncated_normal([latent_space,784],stddev=0.1)),
    'decoder_b':tf.Variable(tf.truncated_normal([784],stddev=0.1)),
}
ae_input = tf.placeholder(tf.float32,[batch_size,784])
noise_input = tf.placeholder(tf.float32,[batch_size,784])

hidden = tf.nn.sigmoid(tf.matmul(noise_input,ae_weights['encoder_w']) + ae_weights['encoder_b'])

noise_hidden = tf.nn.relu(hidden-0.1) + 0.1
noise_visible = tf.nn.sigmoid(
    tf.matmul(noise_hidden, ae_weights['decoder_w']) + ae_weights['decoder_b']
)

deta_rho = tf.reduce_mean(hidden,0)
reg_cost= -tf.reduce_mean(tf.log(deta_rho/rho)*rho + tf.log(1-deta_rho/(1-rho)*(1-rho)))
visible_logits = tf.matmul(hidden, ae_weights['decoder_w']) + ae_weights['decoder_b']
visible = tf.nn.sigmoid(visible_logits)



ae_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits,labels=ae_input))
total_cost = reg_cost + ae_cost
optimizer = tf.train.AdamOptimizer(learning_rate)
ae_op = optimizer.minimize(total_cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        x_batch, _ = mnist.train.next_batch(batch_size)
        noise_mask = np.random.uniform(0.,1.,[batch_size,784]) < 0.3
        noise_batch = x_batch.copy()
        noise_batch[noise_mask] = 0.0
        sess.run(ae_op,feed_dict={ae_input:x_batch,noise_input:noise_batch})
        score = sess.run(ae_cost,feed_dict={ae_input:x_batch,noise_input:noise_batch})
        if i%10==0:
            print(f'score = {score}')
    print(f'score = {score}')


score = 0.6425566673278809
score = 0.4087451100349426
score = 0.3643653392791748
score = 0.28148552775382996
score = 0.273108571767807
score = 0.25276264548301697
score = 0.2478683441877365
score = 0.25839704275131226
score = 0.26120415329933167
score = 0.24736253917217255
score = 0.23616082966327667


Сверточный автоинкодер

In [32]:
batch_size, learning_rate = 64,0.01

ae_weights = {
    "conv": tf.Variable(tf.truncated_normal([5,5,1,4],stddev=0.1)),
    'b_hidden':tf.Variable(tf.truncated_normal([4],stddev=0.1)),
    'deconv':tf.Variable(tf.truncated_normal([5,5,1,4],stddev=0.1)),
    'b_visible':tf.Variable(tf.truncated_normal([1],stddev=0.1))
    
}
input_shape = tf.stack([batch_size,28,28,1])
ae_input = tf.placeholder(tf.float32,[batch_size,784])
images = tf.reshape(ae_input,[-1,28,28,1])
hidden_logits = tf.nn.conv2d(images, ae_weights['conv'],strides=[1,2,2,1],padding='SAME') + ae_weights['b_hidden']

hidden = tf.nn.sigmoid(hidden_logits)
visible_logits = tf.nn.conv2d_transpose(hidden,
                                        ae_weights['deconv'],
                                        input_shape, strides=[1,2,2,1],
                                        padding='SAME'
                                       ) + ae_weights['b_visible']

visible = tf.nn.sigmoid(visible_logits)

optimizer = tf.train.AdamOptimizer(learning_rate)

conv_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits,labels=images))
conv_op = optimizer.minimize(conv_cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        x_batch, _ = mnist.train.next_batch(batch_size)
        sess.run(conv_op,feed_dict={ae_input:x_batch})
        score = sess.run(conv_cost,feed_dict={ae_input:x_batch})
        if i%10==0:
            print(f'score = {score}')
    print(f'score = {score}')


score = 0.5757030844688416
score = 0.3697914481163025
score = 0.27113988995552063
score = 0.2394971400499344
score = 0.21175484359264374
score = 0.21590088307857513
score = 0.18427354097366333
score = 0.19019383192062378
score = 0.1854918748140335
score = 0.1724018156528473
score = 0.16910801827907562
